<a href="https://colab.research.google.com/github/tanzeyl/Self-Study/blob/main/Amazon_Fine_Food_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

In [ ]:
#Connecting and reading the SQLite Database
con = sqlite3.connect("/content/drive/MyDrive/Colab Notebooks/database.sqlite")
filtered_data = pd.read_sql_query("SELECT * from Reviews WHERE Score != 3", con)

In [ ]:
def partition(x):
  if x > 3:
    return "Negative"
  return "Positive"

In [ ]:
actualScore = filtered_data["Score"]
pos_neg = actualScore.map(partition)
filtered_data["Score"] = pos_neg

In [ ]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Negative,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Positive,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Negative,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Positive,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Negative,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
filtered_data.shape

(525814, 10)

In [ ]:
#Removing duplicates
final = filtered_data.drop_duplicates(subset = {"UserId", "ProfileName", "Time", "Text"}, keep="first", inplace = False)

In [ ]:
final.shape

(364173, 10)

In [ ]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

In [ ]:
final.shape

(364171, 10)

# Bag Of Words

In [ ]:
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final["Text"].values)

In [ ]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [ ]:
final_counts.get_shape()

(364171, 115281)

# Text Pre-Processing

In [ ]:
#Find sentences containing HTML tags
import re
i = 0
for sent in final["Text"].values:
  if(len(re.findall("<.*?>", sent ))):
    print(i)
    print(sent)
    break
  i += 1

10
I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!


In [ ]:
import re
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer("english")

def cleanhtml(sentence):
  cleanr = re.compile("<.*?>")
  cleantext = re.sub(cleanr, " ", sentence)
  return cleantext

def cleanpunc(sentence):
  cleaned = re.sub(r'[? | ! | \' |  " | #]', r' ', sentence)
  cleaned = re.sub(r'[. | , | ) | ( | \ | / ]', r' ', cleaned)
  return cleaned

print(stop)
print("******************************")
print(sno.stem("tasty"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'while', 'how', "it's", 'shan', 'between', 'which', 'at', 'him', 'shouldn', "wasn't", 'or', 'theirs', 'doing', 'itself', 'to', 'such', 'm', 'where', 'only', 'the', 'when', "that'll", "isn't", 'here', "don't", 'who', 'over', 'after', 'above', 'doesn', 'has', 'does', 'couldn', 'before', 're', 'am', 'haven', 'having', 'can', 'didn', 'your', 'mustn', 'hers', 'hadn', 'i', 'was', 'our', "couldn't", 'that', "you'll", 'ain', "hadn't", 'wasn', 'been', 'its', 'because', 'it', "mightn't", 'this', 'for', 'his', 'she', 'of', 'some', 'own', 'and', 'you', 'against', 'each', "didn't", 'wouldn', 'just', 'my', 'we', "mustn't", "shan't", 'needn', 'did', 'hasn', 'all', 'off', 'during', 'ma', 'more', 'until', 'd', 'had', "hasn't", 'from', 'both', 'too', 'through', 'ours', 'be', 'y', 'with', 'should', 'so', 'don', 'them', 'myself', 'very', 'isn', 'll', 'yours', "wouldn't", 'than', 'these',

In [ ]:
i = 0
str1 = " "
final_string = []
all_positive_words = []
all_negative_words = []
s = " "
for sent in final["Text"].values:
  filtered_sentence = []
  sent = cleanhtml(sent)
  for w in sent.split():
    for cleaned_words in cleanpunc(w).split():
      if ((cleaned_words.isalpha())  & (len(cleaned_words) > 2)):
        if (cleaned_words.lower() not in stop):
          s = (sno.stem(cleaned_words.lower())).encode("utf8")
          filtered_sentence.append(s)
          if (final["Score"].values)[i] == "Positive":
            all_positive_words.append(s)
          if (final["Score"].values)[i] == "Negative":
            all_negative_words.append(s)
        else:
          continue
      else:
        continue
  str1 = b" ".join(filtered_sentence)
  final_string.append(str1)
  i += 1

In [ ]:
final["CleanedText"] = final_string
#Store final table in SQLite for future references
conn = sqlite3.connect("final.sqlite")
c = conn.cursor()
conn.text_factory = str
final.to_sql("Reviews", conn, schema = None, if_exists = "replace")

In [ ]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Negative,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,b'bought sever vital can dog food product foun...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Positive,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,b'product arriv label jumbo salt peanut peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Negative,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,b'confect around centuri light pillowi citrus ...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Positive,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,b'look secret ingredi robitussin believ found ...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Negative,1350777600,Great taffy,Great taffy at a great price. There was a wid...,b'great taffi great price wide assort yummi ta...


# Bi-grams and N-grams

In [ ]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)
print("Most common positive words: ", freq_dist_positive.most_common(20))
print("Most common negavtive words: ", freq_dist_negative.most_common(20))

Most common positive words:  [(b'tast', 34594), (b'like', 32334), (b'product', 28227), (b'one', 20572), (b'flavor', 19576), (b'would', 18066), (b'tri', 17757), (b'use', 15305), (b'good', 15044), (b'coffe', 14721), (b'get', 13787), (b'buy', 13752), (b'order', 12871), (b'food', 12755), (b'tea', 11662), (b'even', 11088), (b'box', 10844), (b'amazon', 10076), (b'make', 9840), (b'time', 9802)]
Most common negavtive words:  [(b'like', 139429), (b'tast', 129066), (b'good', 112789), (b'flavor', 109641), (b'love', 107396), (b'great', 103925), (b'use', 103891), (b'one', 96737), (b'product', 91054), (b'tri', 86802), (b'tea', 83920), (b'coffe', 78829), (b'make', 75111), (b'get', 72129), (b'food', 64812), (b'would', 55757), (b'time', 55275), (b'buy', 54202), (b'realli', 52716), (b'eat', 52008)]


In [ ]:
count_vect = CountVectorizer(ngram_range = (1, 2))
final_bigram_counts = count_vect.fit_transform(final["Text"].values)

# TF IDF 

In [ ]:
tf_idf_vect = TfidfVectorizer(ngram_range = (1, 2))
final_tf_idf = tf_idf_vect.fit_transform(final["Text"].values)

In [ ]:
final_tf_idf.shape

(364171, 2910192)

In [ ]:
features = tf_idf_vect.get_feature_names()
len(features)

2910192

In [ ]:
features[100000:100010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and']

In [ ]:
#Convert a row in sparse matrix to numpy array
print(final_tf_idf[3, :].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
def top_tfidf_feats(row, features, top_n = 25):
  topn_ids = np.argsort(row)[::-1][:top_n]
  top_feats = [(features[i], row[i]) for i in topn_ids]
  df = pd.DataFrame(top_feats)
  df.columns = {"feature", "tfidf"}
  return df
top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0], features, 25) 

In [ ]:
top_tfidf

,feature,tfidf
0,as jumbo,0.390489
1,jumbo,0.260971
2,unsalted not,0.201475
3,jumbo salted,0.201475
4,vendor intended,0.201475
5,sized unsalted,0.201475
6,arrived labeled,0.187395
7,peanuts,0.186777
8,actually small,0.184594
9,error or,0.176745


# Word2Vec

In [ ]:
import gensim
i = 0
list_of_sent = []
for sent in final["Text"].values:
  filtered_sentence = []
  sent = cleanhtml(sent)
  for w in sent.split():
    for cleaned_words in cleanpunc(w).split():
      if  cleaned_words.isalpha():
        filtered_sentence.append(cleaned_words.lower())
      else:
        continue
    list_of_sent.append(filtered_sentence)

In [ ]:
print(final["Text"].values[0])
print("*******************************")
print(list_of_sent[0])

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
*******************************
['i', 'have', 'bought', 'several', 'of', 'the', 'vitality', 'canned', 'dog', 'food', 'products', 'and', 'have', 'found', 'them', 'all', 'to', 'be', 'of', 'good', 'quality', 'the', 'product', 'looks', 'more', 'like', 'a', 'stew', 'than', 'a', 'processed', 'meat', 'and', 'it', 'smells', 'better', 'my', 'labrador', 'is', 'finicky', 'and', 'she', 'appreciates', 'this', 'product', 'better', 'than', 'most']


In [ ]:
w2v_model = gensim.models.Word2Vec(list_of_sent, min_count = 5, size = 50, workers = 4)

In [ ]:
words = list(w2v_model.wv.vocab)
print(len(words))
w2v_model.wv.most_similar("tasty")